<a href="https://colab.research.google.com/github/haitam-zouhair/workspace_henceforth/blob/master/arabic_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%%shell
cp -r 'drive/My Drive/Projects/Zouhair/' .

In [3]:
import os
os.chdir("Zouhair")

In [6]:
!pip install pyLDAvis

# Dataset

I used [SANAD](https://www.sciencedirect.com/science/article/pii/S2352340919304305) as dataset. Dataset can be downloaded [here](https://data.mendeley.com/datasets/57zpx667y9/2), you can also use directly the dataset from Google Drive, I put them [here](https://drive.google.com/file/d/1hjDR0C5vHPckZisqYby13r2m1i5mtH3p/view?usp=sharing).

In [5]:
%%shell
unzip arabic.zip
unzip Akhbarona.zip -d Akhbarona
unzip Arabiya.zip -d Arabiya
unzip Khaleej.zip -d Khaleej
unzip SANAD_SUBSET.zip -d SANAD_SUBSET
rm *.zip 

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: SANAD_SUBSET/khaleej/Train/Culture/0f8c6c1fbfb1.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/2279dc2b4ef0.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/34c8bdda56b8.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/2a1dd79f213f.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/06e7a2f41572.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/2480530a7c95.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/12ed0be04318.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/37cee96d9287.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/2a4af8ef77b8.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/14bea0b04409.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/2af68aa24de2.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/23c87582fe37.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/279386f5a587.txt  
  inflating: SANAD_SUBSET/khaleej/Train/Culture/178

In [7]:
import pandas as pd


DATANAME = "Arabiya"

data_frame = pd.DataFrame(columns=["text", "label"])

for cat in os.listdir(DATANAME):
    print("reading files from category {}...".format(cat))
    dir_path = os.path.join(DATANAME, cat)

    for file_name in os.listdir(dir_path):
        path = os.path.join(dir_path, file_name)
        
        with open(path, "r") as f:
            text = f.read()

        data_frame = data_frame.append({"text": text, "label": cat},
                                       ignore_index=True)


reading files from category Culture...
reading files from category Tech...
reading files from category Politics...
reading files from category Finance...
reading files from category Medical...
reading files from category Sports...
reading files from category Religion...


In [8]:
print(len(data_frame))
data_frame.head()

71246


,text,label
0,وافق المستشار أحمد الزند، وزير العدل المصري، ع...,Culture
1,عاد الفنان أحمد حلمي للظهور، عبر صفحته الرسمية...,Culture
2,رفض فنانو #تونس قرار الرئيس الأميركي، دونالد #...,Culture
3,"حققت شخصية ""بن شوصان"" في مسلسل ""أبوالملايين"" ت...",Culture
4,"تغيب "" #الخوذ_البيضاء ""، أو الدفاع المدني النا...",Culture


# I/-Arabic Text Processing

Similar to English text, arabic text should be preporcessed before performing any NLP task on it. The pre-processing steps are roughly the same:

* **Tokenization:**
* **Remove Stopwords:** Arabic stop words can be found [here](https://github.com/mohataher/arabic-stop-words/blob/master/list.txt).
* **Lemmatization**:  I am not sure how to lemmatize arabic text, I think in arabic steeming and lemmatization are the same text. I have found this [paper](https://www.aclweb.org/anthology/L18-1181.pdf)  but we need to check this later to be sure (**TO DO**).
* **Stemming**: We use `nltk.SRIStemmer`



In [9]:
%%shell
#Download arabic stop words
wget https://raw.githubusercontent.com/mohataher/arabic-stop-words/master/list.txt
mv list.txt arabic_stop_words.txt

--2020-06-26 00:01:07--  https://raw.githubusercontent.com/mohataher/arabic-stop-words/master/list.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6474 (6.3K) [text/plain]
Saving to: ‘list.txt’

list.txt            100%[===================>]   6.32K  --.-KB/s    in 0s      

2020-06-26 00:01:07 (64.5 MB/s) - ‘list.txt’ saved [6474/6474]



In [10]:
from string import punctuation
from nltk import ISRIStemmer


punctuation += '،؛؟”0123456789“'
stop_words = open("arabic_stop_words.txt").read().splitlines()
stemmer = ISRIStemmer()

def preprocess(text):
    # tokenize
    text = ''.join(c for c in text if c not in punctuation)
    tokens = text.split()  

    # remove stop words   
    tokens = [w for w in tokens if w not in stop_words]

    # stem
    stems = [stemmer.stem(w) for w in tokens]

    return stems


processed_docs = data_frame['text'].map(preprocess)
data_frame['processed_text'] = processed_docs

# II/- Topic modeling using LDA


In [11]:
from gensim import corpora, models

import numpy as np

## 1. Bag of Words

In [12]:
# build bag of words dictionnary
dictionary = corpora.Dictionary(data_frame['processed_text'])

# filter iinfrequent words
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in data_frame['processed_text']]

View Bag of Word result

In [13]:
for i in range(len(bow_corpus[0])):
    print("Word {} (\"{}\") appears {} time.".format(bow_corpus[0][i][0], 
                                               dictionary[bow_corpus[0][i][0]], 
                                               bow_corpus[0][i][1]))

Word 0 ("أحد") appears 1 time.
Word 1 ("أضف") appears 1 time.
Word 2 ("أعضاء") appears 1 time.
Word 3 ("ؤقت") appears 1 time.
Word 4 ("إحل") appears 1 time.
Word 5 ("إظهار") appears 1 time.
Word 6 ("اضة") appears 1 time.
Word 7 ("اهم") appears 1 time.
Word 8 ("بأن") appears 1 time.
Word 9 ("بدل") appears 1 time.
Word 10 ("بصف") appears 1 time.
Word 11 ("بلد") appears 1 time.
Word 12 ("تحدث") appears 1 time.
Word 13 ("تضم") appears 1 time.
Word 14 ("تمز") appears 1 time.
Word 15 ("ثقف") appears 1 time.
Word 16 ("جدد") appears 1 time.
Word 17 ("جلس") appears 3 time.
Word 18 ("جنب") appears 1 time.
Word 19 ("حدد") appears 1 time.
Word 20 ("حصل") appears 1 time.
Word 21 ("حظر") appears 1 time.
Word 22 ("حمد") appears 2 time.
Word 23 ("خبر") appears 1 time.
Word 24 ("خرج") appears 1 time.
Word 25 ("خلف") appears 2 time.
Word 26 ("درة") appears 2 time.
Word 27 ("دكتور") appears 2 time.
Word 28 ("رئس") appears 1 time.
Word 29 ("رجب") appears 1 time.
Word 30 ("رشح") appears 1 time.
Word 31 ("ر

## 2. Running LDA

Our dataset has $6$ topics, we will try to build LDA model with $6$ topics.

In [14]:
NUM_TOPICS = 6

lda_model = models.LdaMulticore(bow_corpus,
                                num_topics=NUM_TOPICS,
                                id2word=dictionary,
                                passes=2,
                                workers=4)

In [15]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} | Word: {}'.format(idx, topic))

Topic: 0 | Word: 0.007*"درس" + 0.006*"عمل" + 0.006*"كثر" + 0.006*"حدث" + 0.005*"علم" + 0.005*"عرض" + 0.005*"بحث" + 0.005*"شخص" + 0.005*"جمع" + 0.005*"امر"
Topic: 1 | Word: 0.011*"نفط" + 0.010*"تحد" + 0.010*"حكم" + 0.009*"دول" + 0.009*"صدر" + 0.008*"رئس" + 0.008*"امر" + 0.007*"قرر" + 0.007*"سعد" + 0.007*"علم"
Topic: 2 | Word: 0.015*"شرك" + 0.009*"عمل" + 0.009*"امر" + 0.008*"جمع" + 0.007*"علم" + 0.007*"عقد" + 0.007*"عرض" + 0.006*"بلغ" + 0.005*"شرع" + 0.005*"رئس"
Topic: 3 | Word: 0.016*"سعد" + 0.010*"حمد" + 0.010*"ندي" + 0.010*"عمل" + 0.009*"قدم" + 0.009*"رئس" + 0.008*"تحد" + 0.008*"لعب" + 0.007*"جمع" + 0.007*"شرك"
Topic: 4 | Word: 0.020*"لعب" + 0.019*"فرق" + 0.013*"هدف" + 0.013*"دور" + 0.012*"برا" + 0.009*"قدم" + 0.008*"بطل" + 0.008*"نخب" + 0.008*"درب" + 0.007*"وسم"
Topic: 5 | Word: 0.016*"شرك" + 0.016*"عمل" + 0.011*"سوق" + 0.009*"نسب" + 0.009*"رفع" + 0.008*"عام" + 0.008*"سعد" + 0.008*"علم" + 0.007*"قطع" + 0.007*"خدم"


Seems that we can recognize some topics from those results

* Topic 0 corresponds to **Finance**
* Topic 1 corresponds to **Sports**
* Topic 2 corresponds to **Culture**
* Topic 3 corresponds to **Tech**   
* Topic 4 correnponds to **Finance**
* Topic 5 corresponds to **Politics**




# III/- Evaluation

## 1) Vizualisation

In [16]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.104142  0.009148       1        1  26.779762
4      0.146232 -0.031578       2        1  23.136461
0      0.035339  0.103152       3        1  14.350277
1     -0.067912 -0.034359       4        1  14.180038
3      0.026255 -0.033322       5        1  14.067597
2     -0.035772 -0.013042       6        1   7.485865, topic_info=     Term          Freq         Total Category  logprob  loglift
424   لعب  80659.000000  80659.000000  Default  30.0000  30.0000
42    شرك  94115.000000  94115.000000  Default  29.0000  29.0000
901   فرق  72595.000000  72595.000000  Default  28.0000  28.0000
5943  نفط  25646.000000  25646.000000  Default  27.0000  27.0000
1331  ندي  38995.000000  38995.000000  Default  26.0000  26.0000
...   ...           ...           ...      ...      ...      ...
132   تحد   3952.726318  53690.496094   Topic6  -5.4044  -0.0167
70    كبر   3818.321777  48265.863281   Topic6  -5.4390   0.0552
219   شهر   3318.050537  37241.707031   Topic6  -5.5795   0.1741
206   سعد   3593.138672  84288.437500   Topic6  -5.4998  -0.5631
578   قدم   3322.589355  68324.523438   Topic6  -5.5781  -0.4314

[573 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
9563       1  0.920365    Co
10220      1  0.886739  FTSE
10220      3  0.032842  FTSE
10220      5  0.032842  FTSE
10220      6  0.032842  FTSE
...      ...       ...   ...
9098       2  0.018413     ‬
9098       3  0.151746     ‬
9098       4  0.008254     ‬
9098       5  0.012698     ‬
9098       6  0.500951     ‬

[2428 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 5, 1, 2, 4, 3])

## 2. Perplexity and Coherence Score

In [17]:
coherence_model_lda = models.CoherenceModel(model=lda_model,
                                     texts=data_frame["processed_text"],
                                     dictionary=dictionary,
                                     coherence='c_v')

print('Coherence Score: {:.2f}'.format(coherence_model_lda.get_coherence()))

Coherence Score: 0.45


# IV/- Content classification using topic modeling

In this section, we use the results of topic modeling in order to build a content classifier. Each document will be represented by a `NUM_TOPICS`-vector capturing the distribution of the learnt topics. This is similar to the approach used in [Phan et al. (2008)](http://gibbslda.sourceforge.net/fp224-phan.pdf)

## 1. Build dataset

In [18]:
N = len(bow_corpus)


X = np.zeros((N, NUM_TOPICS))
y = np.zeros(N, dtype=np.uint8)


label2id = {label: idx for (idx, label)
 in enumerate(list(set(data_frame["label"])))}

for i in range(N):
    topics_proba = lda_model.get_document_topics(bow_corpus[i],
                                              minimum_probability=0.0)
    
    topic_probas = [v for _, v in  topics_proba] 
    X[i,:] = np.array(topic_probas)
    y[i] = label2id[data_frame["label"][i]]
    

## 2. Train classifier

We train some simple classifiers using the extracted feature vectors

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# SVM
svm_clf = SVC()
svm_clf.fit(X_train, y_train)

train_acc = svm_clf.score(X_train, y_train)
test_acc = svm_clf.score(X_test, y_test)
print("SVM: Train accuracy {0:.2f} | Test acuracy {1:.2f}".format(train_acc,
                                                                  test_acc))

SVM: Train accuracy 0.90 | Test acuracy 0.90


# V/- Model Deployment

In this section we save the model, and we load it into `lda_model_uploaded` and test it on new text.



In [ ]:
import warnings
warnings.filterwarnings('ignore')


model_path = "model"

lda_model.save(model_path)
lda_model_loaded = lda_model.load(model_path)

In [ ]:
import pickle

with open("arab_class.pkl", "wb") as f:
    pickle.dump(svm_clf, f, pickle.HIGHEST_PROTOCOL)

In [ ]:

clf = pickle.load(open("arab_class.pkl","rb"))

Consider some text, for example

In [ ]:
#get a random text
text = data_frame.sample()["text"].values[0]

print(text)

# preprocess text
processed_text = preprocess(text)

# get Bag of Words
bow_doc = lda_model_loaded.id2word.doc2bow(processed_text)

# get topics 
topics = lda_model_loaded.get_document_topics(bow_doc,
                                              minimum_probability=0.0)
topic = [v for _, v in  topics] 
query= np.array(topic)

result=clf.predict([query])
A=['Culture', 'Tech', 'Politics', 'Finance', 'Medical', 'Sports']
print(A[result[0]])
print("*"*30)

print([topics])

أعلن في بغداد عن أسماء الأديبات الفائزات بجائزة "نازك الملائكة للإبداع الأدبي" النسوي في دورتها السابعة (دورة 2014)، وتميّزت دورة هذا العام عن الدورات السابقة بسعة المشاركة وتنوّع البلدان؛ حيث تلقت اللجنة التحكيمية 156 مشاركة من داخل وخارج العراق؛ وتناولت المشاركات مواضيع عدة وكلّ ضمن الحقل المختصة فيه. والملاحظة التي يمكن تأشيرها في هذه الدورة هو فوز الأديبات العربيات في مجالي الشعر والقصة؛ تاركات للأديبات العراقيات جوائز النقد. فجائزة الشعر الأولى ذهبت للشاعرة اللبنانية نور حيدر، والثانية للشاعرة السورية ليندا سلمان ابراهيم؛ والجائزة الثالثة للشاعرة المغربية صباح الدبى. أما في مجال القصة القصيرة فكانت الجائزة الأولى للقاصة العراقية إيمان راضي عبد الحسين، والفائزة بالجائزة الثانية القاصة زينة بو رويسة من الجزائر، في الوقت الذي كان المركز الثالث من نصيب القاصة المصرية غادة محمد العبسي. وجاءت جوائز النقد العراقية كالآتي (ماجدة هاتو بالمركز الأول) و(نادية هناوي سعدون بالمركز الثاني) بينما كان المركز الثالث من نصيب (فاطمة بدر حسين). ويذكر أن جائزة نازك الملائكة للإبداع النسوي أسستها وزارة

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} | Word: {}'.format(idx, topic))

In [ ]:
data_frame["label"]


0        Culture
1        Culture
2        Culture
3        Culture
4        Culture
          ...   
71241     Sports
71242     Sports
71243     Sports
71244     Sports
71245     Sports
Name: label, Length: 71246, dtype: object

In [ ]:
result[0]

4

In [ ]:
A

0         Culture
5619         Tech
10029    Politics
14397     Finance
44473     Medical
48188      Sports
Name: label, dtype: object

In [ ]:
A[0]

'Culture'

In [ ]:
! cp "/content/Zouhair/arab_class.pkl" "/content/drive/My Drive/Projects/Zouhair/Models/"

In [ ]:
! cp "/content/Zouhair/model" "/content/drive/My Drive/Projects/Zouhair/Models/"

In [ ]:
! cp "/content/Zouhair/model.expElogbeta.npy" "/content/drive/My Drive/Projects/Zouhair/Models/"

In [ ]:
! cp "/content/Zouhair/model.id2word" "/content/drive/My Drive/Projects/Zouhair/Models/"

In [ ]:
! cp "/content/Zouhair/model.state" "/content/drive/My Drive/Projects/Zouhair/Models/"

In [ ]:
lda_model_loaded = lda_model.load("/content/drive/My Drive/Projects/Zouhair/Models/model")
clf = pickle.load(open("/content/drive/My Drive/Projects/Zouhair/Models/arab_class.pkl","rb"))

#get a random text
text = " هو فريق كرة قدم محترف إسباني أُسس عام 1902، مقره العاصمة الإسبانية مدريد. يلعب الفريق في الدوري الإسباني واختير كأفضل فريق كرة قدم في القرن العشرين، وقد فاز بالدوري الإسباني 33 مرة (رقم قياسي)، وتسعة عشر مرة بكأس ملك إسبانيا وأحرز رقماً قياسياً بحيازته 13 بطولة في دوري أبطال أوروبا "

print(text)

# preprocess text
processed_text = preprocess(text)

# get Bag of Words
bow_doc = lda_model_loaded.id2word.doc2bow(processed_text)

# get topics 
topics = lda_model_loaded.get_document_topics(bow_doc,
                                              minimum_probability=0.0)
topic = [v for _, v in  topics] 
query= np.array(topic)

result=clf.predict([query])
A=['Culture', 'Tech', 'Politics', 'Finance', 'Medical', 'Sports']
print(A[result[0]])
print("*"*30)

print([topics])